In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Albany,42.6001,-73.9662,60.17,25,1,5.01,US,clear sky
1,1,Los Llanos De Aridane,28.6585,-17.9182,67.12,78,0,16.11,ES,clear sky
2,2,Sao Filipe,14.8961,-24.4956,71.91,75,1,9.44,CV,clear sky
3,3,Upernavik,72.7868,-56.1549,-8.18,93,31,12.73,GL,scattered clouds
4,4,Hofn,64.2539,-15.2082,45.61,95,87,10.36,IS,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
6,6,Teluknaga,-6.0989,106.6381,82.18,83,40,4.61,ID,haze
12,12,Atuona,-9.8000,-139.0333,78.33,78,26,14.61,PF,light rain
13,13,Bataipora,-22.2953,-53.2711,75.52,64,41,5.26,BR,scattered clouds
15,15,Pochutla,15.7432,-96.4661,85.82,50,16,4.76,MX,few clouds
16,16,Harper,4.3750,-7.7169,83.37,70,98,8.81,LR,overcast clouds
18,18,Butaritari,3.0707,172.7902,80.44,74,100,3.65,KI,overcast clouds
27,27,Nadapuram,11.7000,75.6667,81.41,84,59,2.37,IN,broken clouds
28,28,Rikitea,-23.1203,-134.9692,77.61,69,0,14.79,PF,light rain
32,32,Itarema,-2.9248,-39.9167,85.82,67,80,6.98,BR,broken clouds
35,35,Puerto Ayora,-0.7393,-90.3518,80.56,81,48,10.47,EC,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                3
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
6,6,Teluknaga,-6.0989,106.6381,82.18,83,40,4.61,ID,haze
12,12,Atuona,-9.8000,-139.0333,78.33,78,26,14.61,PF,light rain
13,13,Bataipora,-22.2953,-53.2711,75.52,64,41,5.26,BR,scattered clouds
15,15,Pochutla,15.7432,-96.4661,85.82,50,16,4.76,MX,few clouds
16,16,Harper,4.3750,-7.7169,83.37,70,98,8.81,LR,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
648,648,Buritizeiro,-17.3511,-44.9622,89.83,41,14,4.90,BR,few clouds
652,652,Mayumba,-3.4320,10.6554,82.33,73,100,3.83,GA,light rain
656,656,Bindura,-17.3019,31.3306,78.15,56,33,3.06,ZW,scattered clouds
660,660,Tabora,-5.0167,32.8000,79.18,56,100,1.95,TZ,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Teluknaga,ID,82.18,haze,-6.0989,106.6381,
12,Atuona,PF,78.33,light rain,-9.8000,-139.0333,
13,Bataipora,BR,75.52,scattered clouds,-22.2953,-53.2711,
15,Pochutla,MX,85.82,few clouds,15.7432,-96.4661,
16,Harper,LR,83.37,overcast clouds,4.3750,-7.7169,
18,Butaritari,KI,80.44,overcast clouds,3.0707,172.7902,
27,Nadapuram,IN,81.41,broken clouds,11.7000,75.6667,
28,Rikitea,PF,77.61,light rain,-23.1203,-134.9692,
32,Itarema,BR,85.82,broken clouds,-2.9248,-39.9167,
35,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.\
hotel_df.replace('',np.nan, inplace=True)
hotel_df.dropna().head


<bound method NDFrame.head of             City Country  Max Temp Current Description      Lat       Lng  \
6      Teluknaga      ID     82.18                haze  -6.0989  106.6381   
12        Atuona      PF     78.33          light rain  -9.8000 -139.0333   
13     Bataipora      BR     75.52    scattered clouds -22.2953  -53.2711   
15      Pochutla      MX     85.82          few clouds  15.7432  -96.4661   
16        Harper      LR     83.37     overcast clouds   4.3750   -7.7169   
..           ...     ...       ...                 ...      ...       ...   
648  Buritizeiro      BR     89.83          few clouds -17.3511  -44.9622   
652      Mayumba      GA     82.33          light rain  -3.4320   10.6554   
656      Bindura      ZW     78.15    scattered clouds -17.3019   31.3306   
660       Tabora      TZ     79.18     overcast clouds  -5.0167   32.8000   
662         Sale      MA     78.64          few clouds  34.0389   -6.8166   

                          Hotel Name  
6     

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))